In [ ]:
import numpy as np
from data import data_video, data_deeplabcut, dataset_opto
from utils import utils
import analysis
import measures
import partial_procrustes_superimposition as pps
import SSM
import os
import pandas as pd
import matplotlib.pyplot as plt
import math
import matplotlib.cm as cm
import pickle
import seaborn as sb
import project

sb.set()

In [ ]:
project_path = r'E:\dlc\OpenfieldOpto'
cfg = project.load_project(project_path)

analysis_path = os.path.join(project_path, 'analysis', 'smooth')

In [ ]:
dataset = dataset_opto.DatasetOpto(cfg=cfg)

## Create/load project

In [ ]:
# project_path = project.create_project(project_path)
project_path = r'E:\dlc\OpenfieldOpto'
cfg = project.load_project(project_path)

analysis_path = os.path.join(project_path, 'analysis', 'smooth')

In [ ]:
dataset = dataset_opto.DatasetOpto(cfg=cfg)

## Choose/calculate reference pose

In [ ]:
# Creates folder with possible reference poses. Then fill in the reference pose of choice in the cfg.yaml file.
# Sessions for reference poses
session_dlc, stim_times = dataset.get_session_dlc('CONTROL', 1248, 1)
session_df = session_dlc.df
session_df = session_df[np.all(session_df.xs('likelihood', level=1, axis=1)>0.9, axis=1)]


In [ ]:
session_df_transl = pps._translate_to_centroid(session_df, cfg['fit_bodyparts'], cfg['impose_bodyparts'])
session_df_transl_rot = pps._rotate_to_xaxis(session_df_transl, cfg['impose_bodyparts'])

In [ ]:
idxs = [500, 1000, 1500, 5000]
fig, axs = plt.subplots(2, 2, figsize=(5, 5), sharex=True)
for i, idx in enumerate(idxs):
    axs[i//2][i%2].scatter(session_df_transl_rot.iloc[idx].xs('x', level=1, axis=0), session_df_transl_rot.iloc[idx].xs('y', level=1, axis=0))
    axs[i//2][i%2].axis('equal')
plt.show()

In [ ]:
mean_pose = np.mean(session_df_transl_rot, axis=0)
filename = os.path.join(cfg['project_path'], 'ref_pose.csv')
pd.DataFrame(mean_pose).T.to_csv(filename)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 3), sharex=True)
ax.scatter(mean_pose.xs('x', level=1, axis=0), mean_pose.xs('y', level=1, axis=0))
ax.axis('equal')
plt.show()

## Preprocess

In [ ]:
analysis.preprocess_all(cfg, dataset, overwrite=False, detect_boxes=False)

## Calculate measures

In [ ]:
analysis.calculate_measures(cfg, dataset)